In [6]:
# Biblotecas de manipulação de dados
import numpy as np
import pandas as pd
# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt
# Testes estatísticos
from scipy.stats import ttest_ind


from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn import datasets, linear_model
from sklearn.metrics import classification_report 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Bibliotecas Locais
from carga import *
from tratamentos import *
from categorizacao import *

pfEnade = doCarga('microdados_enade_2019.txt')
# TRATAMENTOS 
pfEnade = doIdade(pfEnade)
pfEnade = doNota(pfEnade)
pfEnade = doVazios(pfEnade)
pfEnade = doPublicoPrivada(pfEnade)
pfEnade = doHumanasExatas(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doMigracao(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doEscolaridade(pfEnade)
pfEnade = doCategoricos(pfEnade)

In [ ]:
# BomAluno # AJUSTADO
# Publica       -0.22588985181090404, 0.0
# Tempo         -0.06319841811379118 
# BaixaRenda    0.18514620582187313
# QE_I23CAT     -0.16467544491208252  horas dedicou aos estudos
# Exatas        0.08940482401535141
# Familia       -0.09275717382254631
# NU_IDADE      -0.13719553638364812
# CO_REGIAO_CURSO  0.05901777166108786
# Migrado       0.043418457133967366


In [7]:
pfEnade['Publica'].value_counts() 

1    247250
0     84499
Name: Publica, dtype: int64

In [8]:
# CONTAR CLASSES
count_class_0, count_class_1 = pfEnade['Publica'].value_counts()# pfEnade.target.value_counts()
# Divide by class
df_class_0 = pfEnade[pfEnade['Publica'] == 1]   # 1 = Privada
df_class_1 = pfEnade[pfEnade['Publica'] == 0]   # 0 = Publica
df_class_0_under = df_class_0.sample(count_class_1)
dfAjustado = pd.concat([df_class_0_under, df_class_1], axis=0)

dfAjustado['Publica'].value_counts() 

0    84499
1    84499
Name: Publica, dtype: int64

In [20]:
# CARGA - X e y
y = pd.DataFrame(dfAjustado, columns = ['Publica'])
enade_features = [ 'BomAluno','CO_REGIAO_CURSO', 'Jovem', 'Migrado', 'Solo', 'Exatas', 'QE_I23CAT', 'QE_I08CAT', 'Sustento', 'Sexo'] 
# , 'Familia', , 'NU_IDADE' BaixaRenda
# , 'Tempo', 'RecemFormado', 'QE_I08CAT', 'Age'
# Ou solo ou família

X = dfAjustado[enade_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [21]:
modelo_randomforest = RandomForestClassifier()
modelo_randomforest.fit(X_train, y_train.values.ravel())
modelo_randomforest.score(X_test, y_test)

0.6618612157073696

In [22]:
scores = cross_val_score(modelo_randomforest, X_test, y_test.values.ravel(), cv=5)


In [23]:
scores.mean()

0.6502420656266811